In [1]:
import os
import numpy as np
import pandas as pd
from rdkit import Chem
import timeout_decorator
from tqdm.notebook import tqdm

In [2]:
drugtable = pd.read_csv('drug_table_cleaned.csv')
drugtable.head()

,PDB ID,Generic Name,Brand Name,DrugBank ID,ATC Codes,Ligand ID,Target Name,UniProt ID,PDB ID.1,Seq. Identity
0,1QYX,4-Androstenedione,NaN,DB01536,NaN,ASD,Estradiol 17-beta-dehydrogenase 1,P14061,1QYX,89%
1,3RUK,Abiraterone,Zytiga,DB05812,L02BX03,AER,"Steroid 17-alpha-hydroxylase/17,20 lyase",P05093,3RUK,97%
2,4NKV,Abiraterone,Zytiga,DB05812,L02BX03,AER,"Steroid 17-alpha-hydroxylase/17,20 lyase",P05093,4NKV,97%
3,4R1Z,Abiraterone,Zytiga,DB05812,L02BX03,AER,"Steroid 17-alpha-hydroxylase/17,20 lyase",P05093,4R1Z,50%
4,2QMJ,Acarbose,Acarbose#Glucobay#Precose#Prandase,DB00284,A10BF01#A10BD17,"ACR,QPS","Maltase-glucoamylase, intestinal",O43451,2QMJ,99%


## Extract ligands from pdb files and convert to sdf
We parse the dataframe `drugtable`, load the pdb in entry `PDB ID` and extract the ligands with residue IDs in entry `Ligand ID`. Some residues will not be found in the pdb, because we split the rows above and the residue ID belongs to a different pdb entry. 

In [3]:
@timeout_decorator.timeout(20, timeout_exception=TimeoutError)
def extract_ligands(PDBID, residues):
    if os.path.exists(f'pdb/{PDBID}.pdb'):
        mol = Chem.MolFromPDBFile(f'pdb/{PDBID}.pdb', removeHs=False)
        print('   ', mol)
        if mol is not None:
            lig_dict = Chem.rdmolops.SplitMolByPDBResidues(mol, whiteList=residues)
            for res, lig in lig_dict.items():
                lig = Chem.AddHs(lig, addCoords=True)
                Chem.SanitizeMol(lig)
                #Chem.rdmolops.AssignAtomChiralTagsFromStructure(lig)
                Chem.rdmolops.AssignStereochemistry(lig)
                lig.SetProp("PDBID", PDBID)   
                lig_dict[res] = lig
            return lig_dict
        else:
            print(f'Reading {PDBID} failed')
            return {}
    else:
        print(f'File pdb/{PDBID}.pdb does not exist.') 
        return {}

In [4]:
for i, row in tqdm(drugtable.iterrows()):
    PDBID = row["PDB ID"]
    print(PDBID)
    residues = row["Ligand ID"].split(',')
    print('   ', residues)
    try:
        lig_dict = extract_ligands(PDBID, residues)
    except TimeoutError:
        print('Extraction timeout. Continue with next pdb.')
        continue
    print('   ', lig_dict)
    for res, lig in lig_dict.items():
        print('    ', res)
        drugtable.loc[i, "Ligand ID"] = res
        for j, prop in row.iteritems():
            print(j, prop)
            lig.SetProp(j, str(prop))
        drugtable.loc[i, "RDMol"] = lig
        print('    ', res)
    if len(lig_dict) > 1:
        print('More than one ligand.')
drugtable.head()

1QYX
    ['ASD']
    {'ASD': <rdkit.Chem.rdchem.Mol object at 0x7fb6b6be0ad0>}
     ASD
PDB ID 1QYX
Generic Name 4-Androstenedione
Brand Name nan
DrugBank ID DB01536
ATC Codes nan
Ligand ID ASD
Target Name Estradiol 17-beta-dehydrogenase 1
UniProt ID P14061
PDB ID.1 1QYX
Seq. Identity 89%
     ASD
3RUK
    ['AER']
    {'AER': <rdkit.Chem.rdchem.Mol object at 0x7fb6b6be0c10>}
     AER
PDB ID 3RUK
Generic Name Abiraterone
Brand Name Zytiga
DrugBank ID DB05812
ATC Codes L02BX03
Ligand ID AER
Target Name Steroid 17-alpha-hydroxylase/17,20 lyase
UniProt ID P05093
PDB ID.1 3RUK
Seq. Identity 97%
     AER
4NKV
    ['AER']
    {'AER': <rdkit.Chem.rdchem.Mol object at 0x7fb6b6be8e40>}
     AER
PDB ID 4NKV
Generic Name Abiraterone
Brand Name Zytiga
DrugBank ID DB05812
ATC Codes L02BX03
Ligand ID AER
Target Name Steroid 17-alpha-hydroxylase/17,20 lyase
UniProt ID P05093
PDB ID.1 4NKV
Seq. Identity 97%
     AER
4R1Z
    ['AER']
    {'AER': <rdkit.Chem.rdchem.Mol object at 0x7fb6b6bf2a30>}
     AER

    {'AZM': <rdkit.Chem.rdchem.Mol object at 0x7fb6b6c01580>}
     AZM
PDB ID 3CZV
Generic Name Acetazolamide
Brand Name Acetazolam#Acetazolamide#Acetazolamide for Injection, USP#Diamox IV 500mg#Diamox Pws 500mg/vial#Diamox Sequels#Diamox Src 500mg#Diamox Tab 250mg#Diamox Tablets 250mg#Novo-zolamide Tab 250mg#Acemit#Acemox#Acetak#Acetamide#Acetazolamax#Azm#Azol#Azomid#Carbinib#Defiltran#Diabo#Diacarb#Diamox#Diamox Depot#Diazomid#Diluran#Diuramid#Défiltran#Edemox#Glaumox#Glaupax#Glupax#Huma-Zolamide#Iopar-SR#Medene#Oculten#Uramox#Zolmide#Ödemin
DrugBank ID DB00819
ATC Codes S01EC01
Ligand ID AZM
Target Name Carbonic anhydrase 4
UniProt ID P22748
PDB ID.1 3CZV
Seq. Identity 35%
     AZM
1YDB
    ['AZM']
    {'AZM': <rdkit.Chem.rdchem.Mol object at 0x7fb6b6bf2580>}
     AZM
PDB ID 1YDB
Generic Name Acetazolamide
Brand Name Acetazolam#Acetazolamide#Acetazolamide for Injection, USP#Diamox IV 500mg#Diamox Pws 500mg/vial#Diamox Sequels#Diamox Src 500mg#Diamox Tab 250mg#Diamox Tablets 250mg#No

RDKit ERROR: [14:55:17] Explicit valence for atom # 1617 O, 3, is greater than permitted


    {'C41': <rdkit.Chem.rdchem.Mol object at 0x7fb6b6c0cc10>}
     C41
PDB ID 2V0Z
Generic Name Aliskiren
Brand Name Amturnide#Rasilez#Rasilez Hct#Sandoz Aliskiren#Tekamlo#Tekturna#Tekturna Hct
DrugBank ID DB09026
ATC Codes C09DX02#C09XA02#C09XA52#C09XA53#C09XA54
Ligand ID C41
Target Name Renin
UniProt ID P00797
PDB ID.1 2V0Z
Seq. Identity 100%
     C41
1G5Y
    ['9CR']
    {}
3OAP
    ['9CR']
    {'9CR': <rdkit.Chem.rdchem.Mol object at 0x7fb6b6c050d0>}
     9CR
PDB ID 3OAP
Generic Name Alitretinoin
Brand Name Panretin#Toctino
DrugBank ID DB00523
ATC Codes D11AH04#L01XX22
Ligand ID 9CR
Target Name Retinoic acid receptor RXR-beta
UniProt ID P28702
PDB ID.1 3OAP
Seq. Identity 87%
     9CR
1FM6
    ['9CR']
    {'9CR': <rdkit.Chem.rdchem.Mol object at 0x7fb6b6c095d0>}
     9CR
PDB ID 1FM6
Generic Name Alitretinoin
Brand Name Panretin#Toctino
DrugBank ID DB00523
ATC Codes D11AH04#L01XX22
Ligand ID 9CR
Target Name Retinoic acid receptor RXR-beta
UniProt ID P28702
PDB ID.1 1FM6
Seq. Identity

RDKit ERROR: [14:55:18] Explicit valence for atom # 24323 O, 3, is greater than permitted


...

RDKit ERROR: [15:47:42] Explicit valence for atom # 6 O, 3, is greater than permitted


    {'LF7': <rdkit.Chem.rdchem.Mol object at 0x7fb6b6bb92b0>}
     LF7
PDB ID 6B1E
Generic Name Vildagliptin
Brand Name Galvus
DrugBank ID DB04876
ATC Codes A10BH02#A10BD08
Ligand ID LF7
Target Name Dipeptidyl peptidase 4
UniProt ID P27487
PDB ID.1 6B1E
Seq. Identity 100%
     LF7
3W2T
    ['LF7']
    {'LF7': <rdkit.Chem.rdchem.Mol object at 0x7fb6b6ba4490>}
     LF7
PDB ID 3W2T
Generic Name Vildagliptin
Brand Name Galvus
DrugBank ID DB04876
ATC Codes A10BH02#A10BD08
Ligand ID LF7
Target Name Dipeptidyl peptidase 4
UniProt ID P27487
PDB ID.1 3W2T
Seq. Identity 100%
     LF7
5J2T
    ['VLB']
    {'VLB': <rdkit.Chem.rdchem.Mol object at 0x7fb6b6bc3530>}
     VLB
PDB ID 5J2T
Generic Name Vinblastine
Brand Name Velbe 1mg/ml#Vinblastine Sulfate#Vinblastine Sulfate Inj 1mg/ml#Vinblastine Sulfate Injection#Vinblastine Sulphate Injection#Blastivin#Cytoblastin#Lemblastine#Oncostin#Velban#Velbastin#Velbe#Vinblasin#Vinblastin#Vinko#Weibaoding#Xintoprost
DrugBank ID DB00570
ATC Codes L01CA01
Ligan

RDKit ERROR: [15:47:45] Explicit valence for atom # 12672 O, 3, is greater than permitted


    {'ZMR': <rdkit.Chem.rdchem.Mol object at 0x7fb6b6bc5620>}
     ZMR
PDB ID 1A4G
Generic Name Zanamivir
Brand Name Relenza
DrugBank ID DB00558
ATC Codes J05AH01
Ligand ID ZMR
Target Name Neuraminidase
UniProt ID P27907
PDB ID.1 1A4G
Seq. Identity 97%
     ZMR
4CPZ
    ['ZMR']
    {'ZMR': <rdkit.Chem.rdchem.Mol object at 0x7fb6b6bc58f0>}
     ZMR
PDB ID 4CPZ
Generic Name Zanamivir
Brand Name Relenza
DrugBank ID DB00558
ATC Codes J05AH01
Ligand ID ZMR
Target Name Neuraminidase
UniProt ID P27907
PDB ID.1 4CPZ
Seq. Identity 86%
     ZMR
4CPN
    ['ZMR']
    {'ZMR': <rdkit.Chem.rdchem.Mol object at 0x7fb6b6bc55d0>}
     ZMR
PDB ID 4CPN
Generic Name Zanamivir
Brand Name Relenza
DrugBank ID DB00558
ATC Codes J05AH01
Ligand ID ZMR
Target Name Neuraminidase
UniProt ID P27907
PDB ID.1 4CPN
Seq. Identity 86%
     ZMR
3N45
    ['ZOL']
    {'ZOL': <rdkit.Chem.rdchem.Mol object at 0x7fb6b6bc5b70>}
     ZOL
PDB ID 3N45
Generic Name Zoledronic acid
Brand Name Aclasta#Act Zoledronic Acid#Act Zoledron

,PDB ID,Generic Name,Brand Name,DrugBank ID,ATC Codes,Ligand ID,Target Name,UniProt ID,PDB ID.1,Seq. Identity,RDMol
0,1QYX,4-Androstenedione,NaN,DB01536,NaN,ASD,Estradiol 17-beta-dehydrogenase 1,P14061,1QYX,89%,<rdkit.Chem.rdchem.Mol object at 0x7fb6b6be0ad0>
1,3RUK,Abiraterone,Zytiga,DB05812,L02BX03,AER,"Steroid 17-alpha-hydroxylase/17,20 lyase",P05093,3RUK,97%,<rdkit.Chem.rdchem.Mol object at 0x7fb6b6be0c10>
2,4NKV,Abiraterone,Zytiga,DB05812,L02BX03,AER,"Steroid 17-alpha-hydroxylase/17,20 lyase",P05093,4NKV,97%,<rdkit.Chem.rdchem.Mol object at 0x7fb6b6be8e40>
3,4R1Z,Abiraterone,Zytiga,DB05812,L02BX03,AER,"Steroid 17-alpha-hydroxylase/17,20 lyase",P05093,4R1Z,50%,<rdkit.Chem.rdchem.Mol object at 0x7fb6b6bf2a30>
4,2QMJ,Acarbose,Acarbose#Glucobay#Precose#Prandase,DB00284,A10BF01#A10BD17,ACR,"Maltase-glucoamylase, intestinal",O43451,2QMJ,99%,<rdkit.Chem.rdchem.Mol object at 0x7fb6b6bf2ad0>


Remove all rows where RDMOl is NaN

In [5]:
drugtable = drugtable[np.invert(drugtable['RDMol'].isna())]

## Assign a unique identifier to all conformers (based on DrugBank ID) and write to SDF file
The identifier has the format `PDB_DBXXXXX_YY`, where `XXXXX` is part of the DrugBank ID and `YY` is a conformer index starting at 99 and decreasing.

In [6]:
os.makedirs('sdf_crystal', exist_ok=True)
for drug_bank_id in tqdm(drugtable['DrugBank ID'].unique()):
    print(f'PDB_{drug_bank_id}')
    for i, (index, row) in enumerate(drugtable[drugtable['DrugBank ID']==drug_bank_id].iterrows()):
        identifier = f'PDB_{drug_bank_id}_{99-i:02d}'
        drugtable.loc[index, 'identifier'] = identifier
        drugtable.loc[index, 'RDMol'].SetProp('_Name', identifier)
        writer = Chem.SDWriter(f'./sdf_crystal/{identifier}.sdf')
        writer.write(drugtable.loc[index, 'RDMol'])
        writer.close()

PDB_DB01536
PDB_DB05812
PDB_DB00284
PDB_DB00819
PDB_DB00551
PDB_DB00787
PDB_DB08916
PDB_DB11363
PDB_DB08915
PDB_DB09026
PDB_DB00523
PDB_DB00866
PDB_DB00513
PDB_DB00182
PDB_DB00701
PDB_DB00276
PDB_DB06605
PDB_DB00278
PDB_DB01072
PDB_DB01076
PDB_DB01117
PDB_DB06626
PDB_DB00181
PDB_DB06401
PDB_DB05015
PDB_DB00443
PDB_DB00307
PDB_DB01128
PDB_DB08873
PDB_DB00559
PDB_DB06616
PDB_DB01194
PDB_DB01222
PDB_DB00201
PDB_DB00136
PDB_DB01197
PDB_DB00190
PDB_DB01136
PDB_DB00456
PDB_DB00482
PDB_DB09063
PDB_DB01034
PDB_DB01597
PDB_DB03849
PDB_DB00537
PDB_DB00215
PDB_DB05239
PDB_DB00907
PDB_DB01394
PDB_DB00148
PDB_DB08865
PDB_DB04839
PDB_DB08912
PDB_DB01764
PDB_DB01264
PDB_DB01254
PDB_DB00705
PDB_DB06700
PDB_DB00380
PDB_DB00509
PDB_DB08809
PDB_DB00586
PDB_DB01144
PDB_DB00255
PDB_DB00390
PDB_DB00154
PDB_DB00320
PDB_DB00841
PDB_DB00843
PDB_DB00988
PDB_DB00869
PDB_DB00476
PDB_DB01127
PDB_DB01010
PDB_DB00625
PDB_DB06243
PDB_DB00668
PDB_DB00696
PDB_DB00530
PDB_DB00783
PDB_DB04573
PDB_DB00655
PDB_DB00311
PDB_